In [15]:
import pandas as pd
from datetime import timezone
import matplotlib.pyplot as plt
import requests
import json

In [2]:
sensors = pd.read_csv('../DATA/processed/sensors.csv', index_col=0)

In [3]:
sensors.sensor_type.value_counts()

SDS011    246
BME280    171
Name: sensor_type, dtype: int64

In [4]:
sensors.head()

,sensor_id,sensor_type,lat,lon
0,27418,SDS011,55.697960,37.535890
1,21332,SDS011,55.698000,37.498000
2,42870,SDS011,55.887121,37.265067
3,34535,SDS011,55.596000,37.538000
4,43811,SDS011,55.703017,37.459441


In [5]:
sds_sensors = sensors[sensors.sensor_type=='SDS011']
bme_sensors = sensors[sensors.sensor_type=='BME280']

In [6]:
sensors = (sds_sensors.assign(dummy=1)
    .merge(bme_sensors.assign(dummy=1),on='dummy')
    .query('lat_x==lat_y and lon_x==lon_y')
    .drop('dummy', axis=1))[['sensor_id_x', 'sensor_id_y', 'lat_x', 'lon_x']]
sensors = sensors.rename(columns={'sensor_id_x': 'sds_sensor',
                                  'sensor_id_y': 'bme_sensor',
                                  'lat_x': 'lat',
                                  'lon_x': 'lon'})
sensors

,sds_sensor,bme_sensor,lat,lon
833,43811,43812,55.703017,37.459441
947,46280,46281,55.677867,37.456325
1120,33626,33627,55.874089,37.510001
1304,45568,45569,55.625773,37.525268
1508,27213,27214,55.814000,37.460000
...,...,...,...,...
41025,42920,42921,55.850951,37.348591
41050,41097,41098,55.563750,37.644380
41271,48427,48428,55.595953,37.598865
41416,35514,35515,55.650000,37.490000


In [7]:
press_meteo = pd.read_csv('../DATA/processed/dataset.csv', parse_dates=['date']).set_index('date').pres_meteo

In [8]:
press_meteo

date
2019-04-01 00:00:00+00:00     99271.5612
2019-04-01 00:05:00+00:00     99471.5442
2019-04-01 00:10:00+00:00     99471.5442
2019-04-01 00:15:00+00:00     99471.5442
2019-04-01 00:20:00+00:00     99471.5442
                                ...     
2021-01-03 17:40:00+00:00    101324.7200
2021-01-03 17:45:00+00:00    101324.7200
2021-01-03 17:50:00+00:00    101324.7200
2021-01-03 17:55:00+00:00    101324.7200
2021-01-03 18:00:00+00:00    101324.7200
Name: pres_meteo, Length: 185401, dtype: float64

https://ru.wikipedia.org/wiki/%D0%91%D0%B0%D1%80%D0%BE%D0%BC%D0%B5%D1%82%D1%80%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B0%D1%8F_%D1%81%D1%82%D1%83%D0%BF%D0%B5%D0%BD%D1%8C

In [9]:
def get_sealevel(bme_id):
    # see details 
    file_name = f'{bme_id}_bme280_sensor_.csv'
    bme_data = pd.read_csv('../DATA/raw/sensors/'+file_name, sep=';', parse_dates=['timestamp'])
    bme_data = bme_data.set_index('timestamp').resample('5T').mean()
    bme_data = bme_data.tz_localize(timezone.utc)
    bme_data['press_diff'] = (bme_data['pressure'] - press_meteo) / 1000
    bme_data['Q'] = 8000/(bme_data.pressure/1000) * (1 + 0.00366*bme_data.temperature)
    bme_data['delta_h'] = - bme_data.Q * bme_data.press_diff
    delta_h = bme_data.delta_h.median()
    h = delta_h + 125 # высота метеостанции над уровнем моря
    return h
    

In [11]:
sealevels = []
for index, row in sensors.iterrows():
    sealevels.append(get_sealevel(int(row['bme_sensor'])))

[198.44101235495413, 179.95334714151517, 168.125265823983, 263.5806029972, 157.7982775441712, 132.15232774736884, 155.21682591188318, 141.72816514065343, 193.68767526449574, 207.780272906518, 192.20963918461854, 158.70088863474416, 143.38027154604163, 204.7668662336162, 129.06069694991677, 215.13521878573144, 171.9126303863845, 190.70157118511435, 149.27612392744885, 240.99217253818222, 184.05120487634275, 131.59623441553234, 212.16472267174706, 182.4335847300856, 190.8739173208605, 229.29046520374587, 174.53817773335396, 185.18700615210543, 183.75989543112934, 178.85453034540595, 147.49586405929367, 138.7231614759093, 184.06162668111293, 153.34601995200603, 202.64578254774793, 182.02333258831345, 185.42887257107816, 186.52936307235728, 183.9956479398247, 221.69435931320675, 180.30923013601367, 139.9804191053286, 226.32939870450858, 195.13638062557084, 199.17104352690308, 211.86873506805938, 189.80798470979255, 263.3172643572521, 177.60457306008362, 176.47610164373233, 128.373413587004

In [12]:
sensors['sealevel'] = sealevels

In [43]:
import time

surface_levels = []
for index, row in sensors.iterrows():
    lat, lon = row.lat, row.lon
    resp = requests.get(f'https://api.opentopodata.org/v1/srtm30m?locations={lat},{lon}')
    data = json.loads(resp.text)
    surface_levels.append(data['results'][0]['elevation'])
    time.sleep(1)


In [44]:
sensors['surface_alt'] = surface_levels

In [46]:
sensors['surface_level'] = sensors.sealevel - sensors.surface_alt

In [47]:
sensors

,sds_sensor,bme_sensor,lat,lon,sealevel,surface_alt,surface_level
833,43811,43812,55.703017,37.459441,198.441012,163.0,35.441012
947,46280,46281,55.677867,37.456325,179.953347,176.0,3.953347
1120,33626,33627,55.874089,37.510001,168.125266,176.0,-7.874734
1304,45568,45569,55.625773,37.525268,263.580603,230.0,33.580603
1508,27213,27214,55.814000,37.460000,157.798278,152.0,5.798278
...,...,...,...,...,...,...,...
41025,42920,42921,55.850951,37.348591,193.788267,176.0,17.788267
41050,41097,41098,55.563750,37.644380,170.006253,175.0,-4.993747
41271,48427,48428,55.595953,37.598865,195.139890,172.0,23.139890
41416,35514,35515,55.650000,37.490000,222.871522,200.0,22.871522


In [48]:
sensors.to_csv('sensors.csv', index=False)